In [ ]:
import os, glob
import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, matthews_corrcoef,
    roc_curve
)

# ---- Folders and model types to evaluate ----
model_folders = {
    "RF (Proteomics + Demographics)": "/Users/authorname/Desktop/Projects/proteomics_RF(ANML+Meta)",
    "LGBM (Proteomics + Demographics)": "/Users/authorname/Desktop/Projects/proteomics_LGBM(ANML+Meta)",
    "LGBM (Protein Ratios)": "/Users/authorname/Desktop/Projects/proteomics_LGBM(ANML+Meta)_RATIOS_fixed_rfe",
    "LGBM (Protein Ratios + Demographics)": "/Users/authorname/Desktop/Projects/proteomics_LGBM(ANML+Meta)_RATIOS_Demo",
    "RF (Genes + Demographics)": "/Users/authorname/Desktop/Projects/ml4h_project/RF(Genes+Demo)",
    "LGBM (Genes + Demographics)": "/Users/authorname/Desktop/Projects/ml4h_project/LGBM(Genes+Demo)",
    "LGBM (Gene Ratios)": "/Users/authorname/Desktop/Projects/Genomics_LGBM(Genomics+Meta)_RATIOS_fixed_rfe",
    "LGBM (Gene Ratios + Demographics)": "/Users/authorname/Desktop/Projects/Genomics_LGBM(Genomics+Meta)_RATIOS_Demo"
}


# ---- Metrics to compute ----
def compute_metrics(y_true, y_score, y_pred):
    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "F1": f1_score(y_true, y_pred, zero_division=0),
        "ROC AUC": roc_auc_score(y_true, y_score),
        "PR AUC": average_precision_score(y_true, y_score),
        "MCC": matthews_corrcoef(y_true, y_pred),
    }

# ---- Find Youden's J optimal threshold ----
def optimal_threshold_youden(y_true, y_score):
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    youden_j = tpr - fpr
    best_idx = np.argmax(youden_j)
    return thresholds[best_idx]

# ---- Aggregate per-model results ----
rows = []

for model_name, folder in model_folders.items():
    files = glob.glob(os.path.join(folder, "seed*_*.csv"))
    if not files:
        print(f"No files found for {model_name}. Skipping.")
        continue

    # Extract class names
    def extract_class(p): return "_".join(os.path.basename(p).split("_")[1:]).replace(".csv", "")
    class_names = sorted(set(extract_class(f) for f in files))

    for cls in class_names:
        cls_files = sorted(glob.glob(os.path.join(folder, f"seed*_{cls}.csv")))

        metric_lists = {k: [] for k in ["Accuracy", "Precision", "Recall", "F1", "ROC AUC", "PR AUC", "MCC"]}

        for f in cls_files:
            df = pd.read_csv(f)
            y_true = df["y_true"].astype(int).values
            y_score = df["y_score"].astype(float).values

            try:
                # Compute optimal threshold for Youden's J
                opt_thresh = optimal_threshold_youden(y_true, y_score)
                y_pred = (y_score >= opt_thresh).astype(int)

                metrics = compute_metrics(y_true, y_score, y_pred)
                for k, v in metrics.items():
                    metric_lists[k].append(v)
            except Exception as e:
                print(f"Error in {f}: {e}")

        # Aggregate mean ± std
        summary = {
            "Model": model_name,
            "Class": cls
        }
        for k in metric_lists:
            vals = np.array(metric_lists[k])
            summary[f"{k} (mean±std)"] = f"{np.mean(vals):.3f} ± {np.std(vals):.3f}"
        rows.append(summary)

# ---- Final DataFrame ----
summary_df = pd.DataFrame(rows)
summary_df = summary_df.sort_values(by=["Model", "Class"])
summary_df.reset_index(drop=True, inplace=True)

# ---- Save ----
summary_df.to_csv("ML4H_model_eval_summary.csv", index=False)
print(summary_df)

In [ ]:
import pandas as pd

# ---- Load summary CSV ----
df = pd.read_csv("ML4H_model_eval_summary.csv")

# ---- Exact model-name abbreviations (aligned with your model_folders keys) ----
model_map = {
    "RF (Proteomics + Demographics)":              "RF (P + D)",
    "LGBM (Proteomics + Demographics)":            "LGBM (P + D)",
    "LGBM (Protein Ratios)":                        "LGBM (PR)",
    "LGBM (Protein Ratios + Demographics)":         "LGBM (PR + D)",
    "RF (Genes + Demographics)":                    "RF (G + D)",
    "LGBM (Genes + Demographics)":                  "LGBM (G + D)",
    "LGBM (Gene Ratios)":                           "LGBM (GR)",
    "LGBM (Gene Ratios + Demographics)":            "LGBM (GR + D)",
}

# ---- Class abbreviations (as previously specified) ----
class_map = {
    "Typical_AD":         "AD",
    "Healthy_Control":    "Control",
    "Symptomatic_Non-AD": "Symp non-AD",
    "Resilient":          "Resilient",
}

# ---- Apply mappings with checks ----
orig_models = df["Model"].unique().tolist()
orig_classes = df["Class"].unique().tolist()

df["Model_abbr"] = df["Model"].map(model_map)
df["Class_abbr"] = df["Class"].replace(class_map)

# Report any unmapped entries (None/NaN after mapping)
unmapped_models = sorted(set(df.loc[df["Model_abbr"].isna(), "Model"]))
unmapped_classes = sorted(set(df.loc[df["Class_abbr"].isna(), "Class"]))

if unmapped_models:
    print("[WARN] Unmapped models (update model_map):", unmapped_models)
if unmapped_classes:
    print("[WARN] Unmapped classes (update class_map):", unmapped_classes)

# Use original value if not mapped (safety fallback)
df["Model_abbr"] = df["Model_abbr"].fillna(df["Model"])
df["Class_abbr"] = df["Class_abbr"].fillna(df["Class"])

# ---- Keep only the requested columns/metrics ----
cols_out = [
    "Model_abbr",
    "Class_abbr",
    "Accuracy (mean±std)",
    "Precision (mean±std)",
    "Recall (mean±std)",
    "F1 (mean±std)",
    "ROC AUC (mean±std)",
]

df = df[cols_out].copy()
df.rename(columns={
    "Model_abbr": "Model",
    "Class_abbr": "Class",
    "Accuracy (mean±std)": "Accuracy",
    "Precision (mean±std)": "Precision",
    "Recall (mean±std)": "Recall",
    "F1 (mean±std)": "F1",
    "ROC AUC (mean±std)": "AUROC",
}, inplace=True)

# ---- Escape underscores for LaTeX ----
df = df.replace("_", r"\_", regex=True)

# ---- Build LaTeX table ----

# ---- Build LaTeX table ----
rows = []
for _, row in df.iterrows():
    row_str = " & ".join(str(row[c]) for c in ["Model","Class","Accuracy","Precision","Recall","F1","AUROC"]) + r" \\"
    rows.append(row_str)

header_titles = ["Model", "Class", "Accuracy", "Precision", "Recall", "F1", "AUROC"]
col_spec = "l" * len(header_titles)

latex = r"""
\begin{table*}[htbp]
\floatconts
  {tab:eval_core}%
  {\caption{Per-model, per-class evaluation (mean $\pm$ std across 5 seeds).}}%
  {%
  \scriptsize
    \begin{tabular}{""" + col_spec + r"""}
    \toprule
    """ + " & ".join(header_titles) + r""" \\
    \midrule
""" + "\n".join(rows) + r"""
    \bottomrule
    \end{tabular}
    \par
  }
\end{table*}
"""

with open("ML4H_model_eval_summary_table_core.tex", "w") as f:
    f.write(latex)

print(latex)

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc

# --- Folders ---
baseline_folder = "/Users/authorname/Desktop/Projects/proteomics_LGBM(ANML+Meta)"                 # LGBM (Proteomics)
ratios_folder   = "/Users/authorname/Desktop/Projects/proteomics_LGBM(ANML+Meta)_RATIOS_fixed_rfe"  # LGBM (Protein Ratios)

classes = ["MCI", "NCI", "AD", "AD+"]
seeds   = [1, 2, 3, 4, 5]

def safe_cls(c):
    return c.replace("+","plus").replace(" ","_").replace("/","-")

def mean_auc_like_plot(folder):
    """Compute per-class mean AUC exactly like the plotting code:
       for each seed file: fpr,tpr = roc_curve(...); mean auc = mean(auc(fpr,tpr))."""
    mean_by_class = {}
    used_seeds = {}
    for cls in classes:
        aucs, used = [], []
        for seed in seeds:
            path = os.path.join(folder, f"seed{seed}_{safe_cls(cls)}.csv")
            if not os.path.exists(path):
                continue
            df = pd.read_csv(path)
            y_true  = df["y_true"].astype(int).values
            y_score = df["y_score"].astype(float).values
            fpr, tpr, _ = roc_curve(y_true, y_score)  # EXACTLY as in your plot
            aucs.append(auc(fpr, tpr))                # no FPR dedup, no roc_auc_score
            used.append(seed)
        if aucs:
            mean_by_class[cls] = float(np.mean(aucs))
            used_seeds[cls] = used
    return mean_by_class, used_seeds

# Compute means for each folder
base_mean, base_used = mean_auc_like_plot(baseline_folder)
ratio_mean, ratio_used = mean_auc_like_plot(ratios_folder)

# Build table and differences (unpaired means)
common_classes = sorted(set(base_mean) & set(ratio_mean))
rows = []
for cls in common_classes:
    rows.append({
        "Class": cls,
        "base_mean_auc": base_mean[cls],
        "ratio_mean_auc": ratio_mean[cls],
        "diff": ratio_mean[cls] - base_mean[cls],
        "base_seeds": base_used.get(cls, []),
        "ratio_seeds": ratio_used.get(cls, []),
    })

df = pd.DataFrame(rows).sort_values("Class")

# Pretty print (rounding only for display)
fmt = {"base_mean_auc": lambda x: f"{x:.3f}",
       "ratio_mean_auc": lambda x: f"{x:.3f}",
       "diff": lambda x: f"{x:.3f}"}
print(df[["Class","base_mean_auc","ratio_mean_auc","diff"]].to_string(index=False, formatters=fmt))

overall = df["diff"].mean()
print(f"\nAverage ROC AUC difference across classes: {overall:.3f}")

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc

# --- Folders (Genomics) ---
baseline_folder = "/Users/authorname/Desktop/Projects/ml4h_project/LGBM(Genes+Demo)"                 # LGBM (Genomics + Demographics)
ratios_folder   = "/Users/authorname/Desktop/Projects/Genomics_LGBM(Genomics+Meta)_RATIOS_Demo"  # LGBM (Gene Ratios, no demo)

# --- Classes (Genomics labels) ---
classes = ["Healthy Control", "Resilient", "Symptomatic Non-AD", "Typical AD"]
seeds   = [1, 2, 3, 4, 5]

def safe_cls(c):
    return c.replace("+","plus").replace(" ","_").replace("/","-")

def mean_auc_like_plot(folder):
    """Compute per-class mean AUC exactly like the plotting code:
       for each seed file: fpr,tpr = roc_curve(...); mean auc = mean(auc(fpr,tpr))."""
    mean_by_class, used_seeds = {}, {}
    for cls in classes:
        aucs, used = [], []
        for seed in seeds:
            path = os.path.join(folder, f"seed{seed}_{safe_cls(cls)}.csv")
            if not os.path.exists(path):
                continue
            df = pd.read_csv(path)
            y_true  = df["y_true"].astype(int).values
            y_score = df["y_score"].astype(float).values
            fpr, tpr, _ = roc_curve(y_true, y_score)  # EXACTLY as in your plot
            aucs.append(auc(fpr, tpr))                # no FPR dedup, no roc_auc_score
            used.append(seed)
        if aucs:
            mean_by_class[cls] = float(np.mean(aucs))
            used_seeds[cls] = used
    return mean_by_class, used_seeds

# Compute means for each folder
base_mean, base_used = mean_auc_like_plot(baseline_folder)  # Genomics + Demo
ratio_mean, ratio_used = mean_auc_like_plot(ratios_folder)  # Gene Ratios (no demo)

# Build table and differences (unpaired means)
common_classes = sorted(set(base_mean) & set(ratio_mean))
rows = []
for cls in common_classes:
    rows.append({
        "Class": cls,
        "base_mean_auc": base_mean[cls],
        "ratio_mean_auc": ratio_mean[cls],
        "diff": ratio_mean[cls] - base_mean[cls],
        "base_seeds": base_used.get(cls, []),
        "ratio_seeds": ratio_used.get(cls, []),
    })

df = pd.DataFrame(rows).sort_values("Class")

# Pretty print (rounding only for display)
fmt = {"base_mean_auc": lambda x: f"{x:.3f}",
       "ratio_mean_auc": lambda x: f"{x:.3f}",
       "diff": lambda x: f"{x:.3f}"}
print(df[["Class","base_mean_auc","ratio_mean_auc","diff"]].to_string(index=False, formatters=fmt))

overall = df["diff"].mean() if not df.empty else float("nan")
print(f"\nAverage ROC AUC difference across classes: {overall:.3f}")